## 17/12/2020 Update
# Linking and Comparing Local Connect Stats with Low-rank Recurrent Network
* From Local Connect Stats cite[Sharpee]
* From Low-rank structure to EI 

In [11]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pylab as plt
import matplotlib
%matplotlib qt
from numpy import linalg as la
from scipy.optimize import fsolve
from scipy import linalg as scpla
import seaborn as sb
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cmath import *
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import fsolve,leastsq 
from math import tanh,cosh
import math
from sympy import *
from scipy.linalg import schur, eigvals
extras_require = {'PLOT':['matplotlib>=1.1.1,<3.0']},

## Check the results in Sharpee's paper
* local stats, EE,EI,IE,II occupy their own properties, independently
* no pair correlation


In [178]:
Nt=np.array([2000,1000])
NE,NI=Nt[0],Nt[1]
at = Nt/N
N=np.sum(Nt)
aee,aei,aie,aii=4.5,-6.0,5.0,-2.5
JEEsub,JEIsub,JIEsub,JIIsub=aee*np.ones((NE,NE))/NE,aei*np.ones((NE,NI))/NI,aie*np.ones((NI,NE))/NE,aii*np.ones((NI,NI))/NI
J = np.zeros((NE+NI,NE+NI))

# determine gcd
Gmat=np.zeros((2,2))
Gmatorg=np.array([[0.8,1.2],[0.9,0.2]])
# Gmat/=np.sqrt(N)
for i in range(2):
    for j in range(2):
        Gmat[i,j]=Gmatorg[i,j]/np.sqrt(N)
# EE
gij=Gmat[0,0]*np.random.randn(NE,NE)
J[:NE,:NE]=gij.copy()
# EI
gij=Gmat[0,1]*np.random.randn(NE,NI)
J[:NE,NE:]=gij.copy()
# IE
gij=Gmat[1,0]*np.random.randn(NI,NE)
J[NE:,:NE]=gij.copy()
# II
gij=Gmat[1,1]*np.random.randn(NI,NI)
J[NE:,NE:]=gij.copy()

P = np.zeros((N,N))
P[:NE,:NE],P[:NE,NE:],P[NE:,:NE],P[NE:,NE:]=JEEsub.copy(),JEIsub.copy(),JIEsub.copy(),JIIsub.copy()
X = P.copy()+J.copy()

In [113]:
gavg=np.sqrt(np.sum(np.sum(Gmat**2)))/2.0
print('averaging g noise:',gavg*np.sqrt(N))

averaging g noise: 0.8558621384311844


In [105]:
aee,aei,aie,aii=4.5,-6.0,5.0,-2.5
Asmall=np.array([[aee,aei],[aie,aii]])
eigva,_=la.eig(Asmall)
_,sigva,_=la.svd(Asmall)
print('eigenvalue:',eigva,'singular value:',sigva,'eigG',np.sqrt(eigG[0]))

eigenvalue: [1.+4.21307489j 1.-4.21307489j] singular value: [9.12570385 2.05463604] eigG 0.8546614831429354


In [179]:
# all about calculate eigenvalues
eigvrand,eigvecrand=la.eig(J)
eigstruct,eigvecstruct=la.eig(P)
# summation
X = P.copy()+J.copy()
eigvx,eigvecx=la.eig(X)
# theoretical
M = np.zeros((2,2))
for i in range(2):
    for j in range(2):
        M[i,j]=(Gmat[i,j]*np.sqrt(N))**2*(Nt[j]/N)
eigG,eigvecG=la.eig(M)
# low rank small
Asmall=np.array([[aee,aei],[aie,aii]])
eigva,_=la.eig(Asmall)


In [180]:
ticks = np.linspace(-2.0,2.0,3)
lims= [-2.1,2.1]

fig,ax=plt.subplots(figsize=(6,6))
ax.plot(np.sqrt(eigG[0])*np.ones(100),np.linspace(-1.0,1.0,100))
ax.scatter(np.real(eigvrand),np.imag(eigvrand),s=5.0,c='blue',alpha=0.5)
ax.scatter(np.real(eigstruct[:4]),np.zeros(4),s=15.0,c='red',alpha=0.5)
# ax.set_xlim(lims)
# ax.set_ylim(lims)
# ax.set_xticks(ticks)
# ax.set_yticks(ticks)
ax.set_aspect('equal')
ax.grid()

## relate this to low rank (here rank-2)

In [109]:
lvecX,svX,rvecXh=la.svd(X)
rvecX=rvecXh.T
m=lvecX[:,:2].copy()
n=rvecX[:,:2].copy()
n[:,0]*=svX[0]
n[:,1]*=svX[1]

# n[:,1]=rvecX[:,2]*svX[2]
# m[:,1]=lvecX[:,2]

In [111]:
morg,norg=m,n
fig,ax=plt.subplots(4,4,figsize=(9,9))
for i in range(2):
    for j in range(2):
        ax[i][j].scatter(morg[:NE,i],morg[:NE,j],s=5.0,c='red',alpha=0.05)
        ax[i][j].scatter(morg[NE:,i],morg[NE:,j],s=5.0,c='blue',alpha=0.05)
for i in range(2):
    for j in range(2):
        ax[2+i][2+j].scatter(norg[:NE,i],norg[:NE,j],s=5.0,c='red',alpha=0.05)
        ax[2+i][2+j].scatter(norg[NE:,i],norg[NE:,j],s=5.0,c='blue',alpha=0.05)
        
for i in range(2):
    for j in range(2):
        ax[i][2+j].scatter(morg[:NE,i],norg[:NE,j],s=5.0,c='red',alpha=0.05)
        ax[i][2+j].scatter(morg[NE:,i],norg[NE:,j],s=5.0,c='blue',alpha=0.05)
for i in range(2):
    for j in range(2):
        ax[2+i][j].scatter(norg[:NE,i],morg[:NE,j],s=5.0,c='red',alpha=0.05)
        ax[2+i][j].scatter(norg[NE:,i],morg[NE:,j],s=5.0,c='blue',alpha=0.05)
for i in range(4):
    for j in range(4):
        # ax[i][j].set_xlim(lims)
        # ax[i][j].set_ylim(lims)
        # ax[i][j].set_xticks(ticks)
        # ax[i][j].set_yticks(ticks)
        ax[i][j].set_aspect('equal')
        ax[i][j].grid()

In [117]:
# test Dale's law
Nt=np.array([300,100])
NE,NI=Nt[0],Nt[1]
N=np.sum(Nt)
aee,aei,aie,aii=4.5,-6.0,5.0,-2.5
JEEsub,JEIsub,JIEsub,JIIsub=aee*np.ones((NE,NE))/NE,aei*np.ones((NE,NI))/np.sqrt(NE*NI),aie*np.ones((NI,NE))/np.sqrt(NE*NI),aii*np.ones((NI,NI))/NI
P = np.zeros((N,N))
P[:NE,:NE],P[:NE,NE:],P[NE:,:NE],P[NE:,NE:]=JEEsub.copy(),JEIsub.copy(),JIEsub.copy(),JIIsub.copy()
eigvP,_=la.eig(P)
_,sigvp,_=la.svd(P)
Asmall=np.array([[aee,aei],[aie,aii]])
eigva,_=la.eig(Asmall)
_,sigva,_=la.svd(Asmall)

print('eigenvalue:',eigvP[:2],eigva[:2])
print('singular value:',sigvp[:2],sigva[:2])

eigenvalue: [1.+4.21307489j 1.-4.21307489j] [1.+4.21307489j 1.-4.21307489j]
singular value: [9.12570385 2.05463604] [9.12570385 2.05463604]


In [184]:
# test Dale's law
Nt=np.array([100,1000])
NE,NI=Nt[0],Nt[1]
N=np.sum(Nt)
at=Nt/N
# determine gcd
Gmat=np.zeros((2,2))
Gmatorg=np.array([[0.8,1.2],[0.9,0.2]])
# Gmat/=np.sqrt(N)
for i in range(2):
    for j in range(2):
        Gmat[i,j]=Gmatorg[i,j]/np.sqrt(N)
# EE
J=np.zeros((N,N))

gij=np.zeros((NE,NE))
gij=Gmat[0,0]*np.random.randn(NE,NE)
J[:NE,:NE]=gij.copy()
# EI
gij=np.zeros((NE,NI))
gij=Gmat[0,1]*np.random.randn(NE,NI)
J[:NE,NE:]=gij.copy()
# IE
gij=np.zeros((NI,NE))
gij=Gmat[1,0]*np.random.randn(NI,NE)
J[NE:,:NE]=gij.copy()
# II
gij=np.zeros((NI,NI))
gij=Gmat[1,1]*np.random.randn(NI,NI)
J[NE:,NE:]=gij.copy()

gavg=0
for i in range(2):
    for j in range(2):
        gavg+=np.sqrt(Gmatorg[i,j]**2*at[i]*at[j])
M = np.zeros((2,2))
for i in range(2):
    for j in range(2):
        M[i,j]=(Gmatorg[i,j]**2)*at[j]

eigvj,_=la.eig(J)
eigvM,_=la.eig(M)
ticks = np.linspace(-2.0,2.0,3)
lims= [-2.1,2.1]

fig,ax=plt.subplots(figsize=(6,6))
ax.plot(np.sqrt(eigvM[0])*np.ones(100),np.linspace(-1.0,1.0,100))
ax.scatter(np.real(eigvj),np.imag(eigvj),s=5.0,c='blue',alpha=0.5)
print('average eigv:',gavg)


average eigv: 0.858253007850327


In [227]:
# testing distribution of J_{ij}
NE,NI=750,250
sigm,sign=0.5,1.2
mE,nE=sigm*np.random.randn(NE,1)/np.sqrt(NE),sign*np.random.randn(NE,1)/np.sqrt(NE)
JEE = mE@nE.T 
eigv,eigvec=la.eig(JEE)
fig,ax=plt.subplots(figsize=(6,6))
ax.scatter(np.real(eigvj),np.imag(eigvj),s=5.0,c='blue',alpha=0.5)

JEEoff=JEE.copy()
JEEdiag=np.zeros(NE)
for i in range(NE):
    JEEoff[i,i:NE-1]=JEEoff[i,i+1:]
    JEEdiag[i]=JEE[i,i]
fig,ax=plt.subplots(2,1,figsize=(6,6))
Joff=JEEoff[:,:NE-1]
Joff=np.reshape(Joff,(NE**2-NE,1))
n, bins, patches = ax[0].hist(Joff, 100, density=True, alpha=0.5)
n, bins, patches = ax[1].hist(JEEdiag, 100, density=True, alpha=0.5)
print(sigm*sign)



0.6


## What's the distribution of $J_{i,j}$ in $\tilde{\mathbf{X}}=\mathbf{X}+\mathbf{p}$

In [229]:
# testing distribution of J_{ij} in \tilde{X}
NE,NI=2000,250
sigm,sign,grandom=0.6,1.0,0.5
rho=0.0
zetax,zetay=np.random.randn(NE,1),np.random.randn(NE,1)
m = sigm*zetax/np.sqrt(NE)
n =sign*(rho*zetax+np.sqrt(1-rho**2)*zetay)/np.sqrt(NE)
P = m@n.T 
X = (grandom)*np.random.randn(NE,NE)/np.sqrt(NE)

Xtilde=X+P

eigv,eigvec=la.eig(Xtilde)
fig,ax=plt.subplots(figsize=(6,6))
ax.scatter(np.real(eigv),np.imag(eigv),s=5.0,c='blue',alpha=0.5)

JEEoff=Xtilde.copy()
JEEdiag=np.zeros(NE)
for i in range(NE):
    JEEoff[i,i:NE-1]=JEEoff[i,i+1:]
    JEEdiag[i]=Xtilde[i,i]
fig,ax=plt.subplots(2,1,figsize=(6,6))
Joff=JEEoff[:,:NE-1]
Joff=np.reshape(Joff,(NE**2-NE,1))
n, bins, patches = ax[0].hist(Joff, 100, density=True, alpha=0.5)
n, bins, patches = ax[1].hist(JEEdiag, 100, density=True, alpha=0.5)

## Generate Random Block Matrix, embedding low rank structur $\tilde{\mathbf{X}}_{\mathrm{type}}=\mathbf{X}_{\mathrm{type}}+\mathbf{P}_{\mathrm{type}}$

In [238]:
phiacttt=np.zeros((N,1))
phiacttt[:NE,0],phiacttt[NE:,0]=(rhoE*zetaxE+np.sqrt(1-rhoE**2)*zetayE)/np.sqrt(N),(rhoI*zetaxI+np.sqrt(1-rhoI**2)*zetayI)/np.sqrt(N)
eitatt=Xtilde@phiacttt
eitarand=X@phiacttt
coveitatt=eitatt@eitatt.T 
coveitarand=eitarand@eitarand.T 
deltaEt,deltaEr=coveitatt[:NE,:NE],coveitarand[:NE,:NE]
deltaIt,deltaIr=coveitatt[NE:,NE:],coveitarand[NE:,NE:]
print('std, Et:',np.std(deltaEt),'; random:',np.std(deltaEr))
print('std, It:',np.std(deltaIt),'; random:',np.std(deltaIr))

In [411]:
eigvx,_=la.eig(X)
eigvxt,_=la.eig(Xtilde)
fig,ax=plt.subplots(2,2,figsize=(9,9))
ax[0][0].scatter(np.real(eigvx),np.imag(eigvx),s=2.0,c='red',alpha=0.5)
ax[0][0].set_aspect('equal')
print(np.sqrt(xee**2*NE/N))
ax[1][0].scatter(np.real(eigvxt),np.imag(eigvxt),s=2.0,c='red',alpha=0.5)
ax[1][0].set_aspect('equal')
# HISTOGRAM
n, bins, patches = ax[0][1].hist(np.real(eigvx), 20, density=True, alpha=0.5)
ax[0][1].set_xlim([-4.0,4.0])
n, bins, patches = ax[1][1].hist(np.real(eigvxt), 20, density=True, alpha=0.5)
ax[0][1].set_xlim([-4.0,4.0])
# SUBTYPE
XEI,XTEI=X[:NE,NE:],Xtilde[:NE,NE:]
eigvEI,eigvec=la.eig(XEI)
eigvTEI,eigvec=la.eig(XTEI)
fig,ax=plt.subplots(1,2,figsize=(6,6))
ax[0].scatter(np.real(eigvEI),np.imag(eigvEI),s=5.0,c='blue',alpha=0.5)
ax[1].scatter(np.real(eigvTEI),np.imag(eigvTEI),s=5.0,c='blue',alpha=0.5)

XEE,XTEE=X[:NE,:NE],Xtilde[:NE,:NE]
eigvEE,eigvec=la.eig(XEE)
eigvTEE,eigvec=la.eig(XTEE)
fig,ax=plt.subplots(1,2,figsize=(6,6))
ax[0].scatter(np.real(eigvEE),np.imag(eigvEE),s=5.0,c='red',alpha=0.5)
ax[1].scatter(np.real(eigvTEE),np.imag(eigvTEE),s=5.0,c='red',alpha=0.5)

XIE,XTIE=X[NE:,:NE],Xtilde[NE:,:NE]
eigvIE,eigvec=la.eig(XIE)
eigvTIE,eigvec=la.eig(XTIE)
fig,ax=plt.subplots(1,2,figsize=(6,6))
ax[0].scatter(np.real(eigvIE),np.imag(eigvIE),s=5.0,c='blue',alpha=0.5)
ax[1].scatter(np.real(eigvTIE),np.imag(eigvTIE),s=5.0,c='blue',alpha=0.5)

XII,XTII=X[NE:,NE:],Xtilde[NE:,NE:]
eigvII,eigvec=la.eig(XII)
eigvTII,eigvec=la.eig(XTII)
fig,ax=plt.subplots(1,2,figsize=(6,6))
ax[0].scatter(np.real(eigvII),np.imag(eigvII),s=5.0,c='red',alpha=0.5)
ax[1].scatter(np.real(eigvTII),np.imag(eigvTII),s=5.0,c='red',alpha=0.5)


1.7677669529663689


# 24/12 Generate Figures and Some Questions

In [3]:
def calculate_angle(vector_1,vector_2):
    unit_vector_1 = vector_1 /np.linalg.norm(vector_1)
    unit_vector_2 = vector_2 /np.linalg.norm(vector_2)
    dot_product = np.dot(unit_vector_1, unit_vector_2)
    angle = np.arccos(dot_product)
    return angle

## Generate Matrix

In [1558]:
# Block Gaussian Random Network
xee,xei,xie,xii=2.0,0.0,0.0,1.5#1.2,1.2,1.8
JE,JI=2.5,2.0

Niter =5
Np = 20
xees=np.linspace(0.4,2.4,Np)
gaverages=np.zeros((Np,Niter))
gradiuss=np.zeros((Np,Niter))
svams = np.zeros((Np,Niter))
svxts = np.zeros((Np,Niter))
for i in range(Niter):
    for idx, xee in enumerate(xees):
        # generate random 
        Grand=np.array([xee,0,0,xii])
        X,Gamp,eigvgm,eigvecgm,gaverages[idx,i],ugm,svgm,vgm=generate_localstatsmat(Nparams,Grand)
        gradiuss[idx,i]=np.sqrt(eigvgm[0])
        # generate mean Am
        Am,Jsv,us,svams[idx,i],va=generate_meanmat(Nparams,JE,JI)
        # CALCULATE XTOTAL
        Xtotal=X.copy()+Am.copy()
        # Combine, total svd decomposition
        uxt,svxt,vhxt=la.svd(Xtotal)
        vxt=vhxt.T   
        svxts[idx,i]=svxt[0]





In [1560]:
ticks = np.linspace(4.7,5.5,3)
lims= [4.69,5.51]
# calculate the theoretical singular values of Xtotal
svxts_theo = np.mean(svams[0]+gaverages**2/svams[0],axis=1)
svxts_theo_eff = np.mean(svams[0]+gradiuss**2/svams[0],axis=1)
svxtsm=np.mean(svxts,axis=1)
fig,ax=plt.subplots(figsize=(6,6))
ax.plot(svxtsm,svxtsm,'k--',linewidth=1.0,alpha=0.5)
ax.plot(svxtsm,svxts_theo,'r.',linewidth=1.5)
ax.plot(svxtsm,svxts_theo_eff,'b.',linewidth=1.5)
ax.set_xlim(lims)
ax.set_ylim(lims)
ax.set_xticks(ticks)
ax.set_yticks(ticks)
ax.set_aspect('equal')




In [2263]:
# Block Gaussian Random Network
xee,xei,xie,xii=0.8,0.5,0.5,0.6#1.2,1.2,1.8
JE,JI=2.0,4.0

# generate random 
Grand=np.array([xee,xei,xie,xii])
X,Gamp,eigvgm,eigvecgm,gaverage,ugm,svgm,vgm=generate_localstatsmat(Nparams,Grand)
gradius=np.sqrt(eigvgm[0])
# generate mean Am
Am,Jsv,ua,svam,va=generate_meanmat(Nparams,JE,JI)
# CALCULATE XTOTAL
Xtotal=X.copy()+Am.copy()
# Combine, total svd decomposition
uxt,svxt,vhxt=la.svd(Xtotal)
vxt=vhxt.T   


In [2264]:
## difference between singular value and eigenvalue
# First Set of Figures
# figure, validate results in sharpee's paper
# ticks and labels
# eigvx,eigvecx=la.eig(Xtotal)
theta = np.linspace(0, 2 * np.pi, 200)
xc = gradius*np.cos(theta)
yc = gradius*np.sin(theta)

xa = gaverage*np.cos(theta)
ya = gaverage*np.sin(theta)

ticks = np.linspace(-1.50,1.50,3)
lims= [-1.51,1.51]
nbins=100
fig,ax=plt.subplots(figsize=(6,6))
# eigenvalue
ax.scatter(np.real(eigvx),np.imag(eigvx),s=2.0,c='blue',alpha=0.5)
ax.plot(xc, yc, color="darkred", linewidth=2)
ax.plot(xa, ya, color="grey", linewidth=2,linestyle='--')
ax.set_xlim(lims)
ax.set_ylim(lims)
ax.set_xticks(ticks)
ax.set_yticks(ticks)
ax.set_aspect('equal')
ax.set_title(r'$J_E,\ J_I=$'+str(JE)+','+str(JI)+'\n $\lambda_1=$'+str(format(gradius,'.1f'))+r', $g_{avg}=$'+str(format(gaverage,'.1f')),fontsize=14)
print(np.sqrt(0.5*JE**2+0.5*JI**2),np.min(np.real(eigvx)),gradius)
print(gaverage**2/svam+svam,svxt[0],gradius**2/svam+svam)

ValueError: operands could not be broadcast together with shapes (2,) (200,) 

Parameters
a(M, M) array_like
Matrix to decompose
output{‘real’, ‘complex’}, optional
Construct the real or complex Schur decomposition (for real matrices).
lworkint, optional
Work array size. If None or -1, it is automatically computed.
overwrite_abool, optional
Whether to overwrite data in a (may improve performance).
sort{None, callable, ‘lhp’, ‘rhp’, ‘iuc’, ‘ouc’}, optional
Specifies whether the upper eigenvalues should be sorted. A callable may be passed that, given a eigenvalue, returns a boolean denoting whether the eigenvalue should be sorted to the top-left (True). Alternatively, string parameters may be used:
'lhp'   Left-hand plane (x.real < 0.0)
'rhp'   Right-hand plane (x.real > 0.0)
'iuc'   Inside the unit circle (x*x.conjugate() <= 1.0)
'ouc'   Outside the unit circle (x*x.conjugate() > 1.0)

## test how the iid vectors constitute overall
## put them together and try to figure out the the properties

In [12]:
Nt=np.array([1000,1000])
NE,NI=Nt[0],Nt[1]
Nparams=np.array([NE,NI])

def generate_localstatsmat(Nparams,Grand):
    xee,xei,xie,xii=Grand[0],Grand[1],Grand[2],Grand[3]
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    at = Nt/N   
    X=np.zeros((N,N))
    X[:NE,:NE],X[NE:,NE:]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    X[:NE,NE:],X[NE:,:NE]=np.random.randn(NE,NE),np.random.randn(NE,NE)

    X[:NE,:NE],X[NE:,NE:]=xee*X[:NE,:NE]/np.sqrt(NE),xii*X[NE:,NE:]/np.sqrt(NI)
    X[:NE,NE:],X[NE:,:NE]=xei*X[:NE,NE:]/np.sqrt(NE),xie*X[NE:,:NE]/np.sqrt(NI)
    # print('std indeed:',xee/np.sqrt(N),xei/np.sqrt(N),xie/np.sqrt(N),xii/np.sqrt(N))
    eigvx,eigvecx=la.eig(X)
    # theoretical M (grandom) matrix 
    gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
    # first do not multiply at
    gaverage=0
    for i in range(2):
        for j in range(2):
            gaverage+=gmat[i,j]/2 # ntype=2
    gaverage=np.sqrt(gaverage)
    # for i in range(2):
    #     gmat[:,i]*=at[i]
    eigvgm,eigvecgm=schur(gmat)
    # eigvgm=eigvals(eigvgm)
    ugm,svgm,vhgm=la.svd(gmat)
    vgm=vhgm.T
    ngm=vhgm.copy()
    ngm=ngm.T 
    ngm[:,0],ngm[:,1]=svgm[0]*ngm[:,0],svgm[1]*ngm[:,1]

    # properties
    Gamp=np.zeros((2,2))
    Gamp[0,0],Gamp[0,1],Gamp[1,0],Gamp[1,1]=xee,xei,xie,xii

    return (X,Gamp,eigvgm,eigvecgm,gaverage,ugm,svgm,vgm)

def generate_meanmat(Nparams,JEE,JIE,JEI,JII):
    # mean value 
    # first use rank-1 structure
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    at = Nt/N   
    Am=np.zeros((N,N))
    Am[:NE,:NE],Am[:NE,NE:]=JEE/NE,-JEI/NI
    Am[NE:,:NE],Am[NE:,NE:]=JIE/NE,-JII/NI
    ua,svam,vha=la.svd(Am)
    va=vha.T
    svam=svam[0]
    ua,va=np.zeros((N,N)),np.zeros((N,N))
    # ua[:,0]=1.0/np.sqrt(N)
    # va[:NE,0]=JE/np.sqrt(NE)
    # va[NE:,0]=-JI/np.sqrt(NI)
    # va[:,0]/=-np.linalg.norm(va[:,0])
    # svam=np.sqrt(2*(JE**2+JI**2))
    # vha=va.T

    Jsv=np.zeros((2,2))
    Jsv[0,0],Jsv[0,1],Jsv[1,0],Jsv[1,1]=JE,JI,JE,JI

    return (Am,Jsv,ua,svam,va)





In [160]:
# Block Gaussian Random Network
xee,xei,xie,xii=0.5,0.0,0.0,0.6#1.2,1.2,1.8
JE,JI=9.0*xee,10.0*xii
# properties
Gamp=np.zeros((2,2))
Gamp[0,0],Gamp[0,1],Gamp[1,0],Gamp[1,1]=xee,xei,xie,xii
Gamp=Gamp**2
eigvG,eigvecG=la.eig(Gamp)
Gampnew=eigvecG.T@Gamp@eigvecG
J=np.zeros((2,2))
J[:,0],J[:,1]=JE,-JI 
Jnew=eigvecG.T@J@eigvecG
# print(Jnew)
xee,xei,xie,xii=np.sqrt(Gampnew[0,0]),np.sqrt(Gampnew[0,1]),np.sqrt(Gampnew[1,0]),np.sqrt(Gampnew[1,1])
# print(xee,xei,xie,xii,Gampnew)
JEE,JIE,JEI,JII=Jnew[0,0],Jnew[1,0],-Jnew[0,1],-Jnew[1,1]


# generate random 
Grand=np.array([xee,0,0,xii])
X,Gamp,eigvgmT,eigvecgm,gaverage,ugm,svgm,vgm=generate_localstatsmat(Nparams,Grand)
# print(Gamp[0,0],Gamp[1,1])
eigvgm=eigvals(eigvgmT)
# print(eigvgm)
idx = eigvgm.argsort()[::-1]   
eigvgm = eigvgm[idx]
gradius=np.sqrt(eigvgm[0])
# generate mean Am
Am,Jsv,ua,svam,va=generate_meanmat(Nparams,JEE,JIE,JEI,JII)


In [161]:
Xtotal=X.copy()+Am.copy()
# singular decomposition blockage
XEEtotal,XEItotal,XIEtotal,XIItotal=Xtotal[:NE,:NE],Xtotal[:NE,NE:],Xtotal[NE:,:NE],Xtotal[NE:,NE:]
uxtEE,svxtEE,vhxtEE=la.svd(XEEtotal)
vxtEE=vhxtEE.T
uxtEI,svxtEI,vhxtEI=la.svd(XEItotal)
vxtEI=vhxtEI.T
uxtIE,svxtIE,vhxtIE=la.svd(XIEtotal)
vxtIE=vhxtIE.T
uxtII,svxtII,vhxtII=la.svd(XIItotal)
vxtII=vhxtII.T
# Combine, total svd decomposition
uxt,svxt,vhxt=la.svd(Xtotal)
vxt=vhxt.T
# decomposition of the noise
ux,svx,vhx=la.svd(X)
vx=vhx.T
# print singular values
print('singular value, outliers')
print(svxt[:3])
print(svx[:3])

singular value, outliers
[10.62649089  1.18943133  1.18037922]
[1.18969155 1.18043338 1.17840104]


## plot eigenvalues here

In [162]:
eigvx,_=la.eig(Xtotal)
theta = np.linspace(0, 2 * np.pi, 200)
xc = gradius*np.cos(theta)
yc = gradius*np.sin(theta)

xa = gaverage*np.cos(theta)
ya = gaverage*np.sin(theta)

tickeig=np.fabs(JE-JI)*1.5
ticks = np.linspace(-tickeig,tickeig,3)
lims= [-tickeig-0.01,tickeig+0.01]
nbins=100
fig,ax=plt.subplots(figsize=(6,6))
# eigenvalue
ax.scatter(np.real(eigvx),np.imag(eigvx),s=2.0,c='blue',alpha=0.5)
ax.plot(xc, yc, color="darkred", linewidth=2)
ax.plot(xa, ya, color="grey", linewidth=2,linestyle='--')
ax.set_xlim(lims)
ax.set_ylim(lims)
ax.set_xticks(ticks)
ax.set_yticks(ticks)
ax.set_aspect('equal')
ax.set_title(r'$J_{EE},J_{IE}\ J_{EI},J_{II}=$'+str(format(JEE,'.2f'))+','+str(format(JIE,'.2f'))+','+str(format(JEI,'.2f'))+','+str(format(JII,'.2f'))+'\n $\lambda_1=$'+str(format(gradius,'.1f'))+r', $g_{avg}=$'+str(format(gaverage,'.1f')),fontsize=14)
# print(np.sqrt(0.5*JE**2+0.5*JI**2),np.min(np.real(eigvx)),gradius)
# print(gaverage**2/svam+svam,svxt[0],gradius**2/svam+svam)
print(Jnew)

[[ 4.5 -6. ]
 [ 4.5 -6. ]]


In [137]:
print(Jnew[0,0]/Jnew[1,0],Jnew[0,1]/Jnew[1,1])
print(Jnew[0,0]/Jnew[0,1],Jnew[1,0]/Jnew[1,1])

0.10870026769825601 0.1087002676982559
-2.871878997200212 -2.871878997200209


In [163]:
iid_sv=np.zeros((2,2))
iid_sv=np.array([[JEE,JEI],[JIE,JII]])
iid_sv[0,0]=iid_sv[0,0]+xee**2/iid_sv[0,0]
iid_sv[0,1]=iid_sv[0,1]+xei**2/iid_sv[0,1]
iid_sv[1,0]=iid_sv[1,0]+xie**2/iid_sv[1,0]
iid_sv[1,1]=iid_sv[1,1]+xii**2/iid_sv[1,1]



In [164]:
## test EE, EI, IE, II separately
iid_mu_u,iid_mu_v ={},{}

iid_mu_u['E'],iid_mu_u['I']=np.zeros((2,1)),np.zeros((2,1))
iid_mu_v['E'],iid_mu_v['I']=np.zeros((2,1)),np.zeros((2,1))

iid_mu_u['E'][0,0],iid_mu_u['E'][1,0]=np.mean(uxtEE[:,0]),np.mean(uxtEI[:,0])
iid_mu_u['I'][0,0],iid_mu_u['I'][1,0]=np.mean(uxtIE[:,0]),np.mean(uxtII[:,0])
iid_mu_v['E'][0,0],iid_mu_v['E'][1,0]=np.mean(vxtEE[:,0]),np.mean(vxtIE[:,0])
iid_mu_v['I'][0,0],iid_mu_v['I'][1,0]=np.mean(vxtEI[:,0]),np.mean(vxtII[:,0])

mua,mva = np.zeros((2,1)),np.zeros((2,1))
mua[0,0],mua[1,0]=ua[0,0],ua[NE+1,0]
mva[0,0],mva[1,0]=va[0,0],va[NE+1,0]

iid_noise_u,iid_noise_v ={},{}
iid_noise_u['E'],iid_noise_u['I']=np.zeros((NE,2)),np.zeros((NE,2))
iid_noise_v['E'],iid_noise_v['I']=np.zeros((NE,2)),np.zeros((NE,2))

iid_noise_u['E'][:,0],iid_noise_u['E'][:,1]=(uxtEE[:,0]-iid_mu_u['E'][0,0]),(uxtEI[:,0]-iid_mu_u['E'][1,0])
iid_noise_u['I'][:,0],iid_noise_u['I'][:,1]=(uxtIE[:,0]-iid_mu_u['I'][0,0]),(uxtII[:,0]-iid_mu_u['I'][1,0])
iid_noise_v['E'][:,0],iid_noise_v['E'][:,1]=(vxtEE[:,0]-iid_mu_v['E'][0,0]),(vxtIE[:,0]-iid_mu_v['E'][1,0])
iid_noise_v['I'][:,0],iid_noise_v['I'][:,1]=(vxtEI[:,0]-iid_mu_v['I'][0,0]),(vxtII[:,0]-iid_mu_v['I'][1,0])
# std noise iid
std_iidn_u =np.zeros((2,2))
std_iidn_u[0,0],std_iidn_u[0,1]=np.std(iid_noise_u['E'][:,0])*np.sqrt(NE),np.std(iid_noise_u['E'][:,1])*np.sqrt(NE)
std_iidn_u[1,0],std_iidn_u[1,1]=np.std(iid_noise_u['I'][:,0])*np.sqrt(NE),np.std(iid_noise_u['I'][:,1])*np.sqrt(NE)

# std noise iid v
std_iidn_v =np.zeros((2,2))
std_iidn_v[0,0],std_iidn_v[0,1]=np.std(iid_noise_v['E'][:,0])*np.sqrt(NE),np.std(iid_noise_v['E'][:,1])*np.sqrt(NE)
std_iidn_v[1,0],std_iidn_v[1,1]=np.std(iid_noise_v['I'][:,0])*np.sqrt(NE),np.std(iid_noise_v['I'][:,1])*np.sqrt(NE)
# print(std_iidn_v)

mu_u,mu_v=np.zeros((2,1)),np.zeros((2,1))
mu_u[0,0],mu_u[1,0]=np.mean(uxt[:NE,0]),np.mean(uxt[NE:,0])
mu_v[0,0],mu_v[1,0]=np.mean(vxt[:NE,0]),np.mean(vxt[NE:,0])

## print noise tt
noise_u,noise_v=np.zeros((NE,2)),np.zeros((NE,2))
noise_u[:,0],noise_u[:,1]=(uxt[:NE,0]-mu_u[0,0]),(uxt[NE:,0]-mu_u[1,0])
noise_v[:,0],noise_v[:,1]=(vxt[:NE,0]-mu_v[0,0]),(vxt[NE:,0]-mu_v[1,0])

# std noise total
std_noise_u =np.zeros((2,1))
std_noise_u[0,0],std_noise_u[1,0]=np.std(noise_u[:,0])*np.sqrt(NE),np.std(noise_u[:,1])*np.sqrt(NE)
std_noise_v =np.zeros((2,1))
std_noise_v[0,0],std_noise_v[1,0]=np.std(noise_v[:,0])*np.sqrt(NE),np.std(noise_v[:,1])*np.sqrt(NE)


## testing the simplest version

In [106]:
tavgsv=svam+gaverage**2/svam
tmaxgsv=svam+eigvgm[0]**2/svam
print(svxt[0],tavgsv,tmaxgsv)
print('test noise:',xee/JE/np.sqrt(NE),np.std(iid_noise_u['E'][:,0]))
print('test noise:',xii/JI/np.sqrt(NE),np.std(iid_noise_u['I'][:,1]))

print('test noise:',xie/JE/np.sqrt(NE),np.std(iid_noise_u['I'][:,0]))
print('test noise:',xei/JI/np.sqrt(NE),np.std(iid_noise_u['E'][:,1]))

33.72617014504863 33.67201394157806 (33.79978154949255+0j)
test noise: 0.003162277660168379 0.0032509050269619024
test noise: 0.0026352313834736496 0.002514968830579337
test noise: 0.0 1.5045170264745657e-16
test noise: 0.0 3.1143063813321916e-17


## the complete version

In [165]:
tavgsv=svam+gaverage**2/svam
tmaxgsv=svam+eigvgm[0]**2/svam
print(svxt[0],tavgsv,tmaxgsv)
print('test noise:',xee/JEE/np.sqrt(NE),np.std(iid_noise_u['E'][:,0]))
print('test noise:',xii/JII/np.sqrt(NE),np.std(iid_noise_u['I'][:,1]))

print('test noise:',xie/JIE/np.sqrt(NE),np.std(iid_noise_u['I'][:,0]))
print('test noise:',xei/JEI/np.sqrt(NE),np.std(iid_noise_u['E'][:,1]))

10.626490892509832 10.635357393566442 (10.618820522977094+0j)
test noise: 0.0035136418446315323 0.003599229570692584
test noise: 0.003162277660168379 0.0031285276751327764
test noise: 0.0 1.153609265197698e-16
test noise: 0.0 2.829193825294734e-17


## integrate all iid noise base and tilde noise base

In [166]:
# u base
unoisebase=np.zeros((NE,4))
unoisebase[:,0]=iid_noise_u['E'][:,0]/std_iidn_u[0,0]
unoisebase[:,3]=iid_noise_u['I'][:,1]/std_iidn_u[1,1]

# v base
vnoisebase=np.zeros((NE,4))
vnoisebase[:,0]=iid_noise_v['E'][:,0]/std_iidn_v[0,0]
vnoisebase[:,3]=iid_noise_v['I'][:,1]/std_iidn_v[1,1]

# # u base
# unoisebase=np.zeros((NE,4))
# unoisebase[:,0]=iid_noise_u['E'][:,0]/std_iidn_u[0,0]
# unoisebase[:,1]=iid_noise_u['E'][:,1]/std_iidn_u[0,1]#*0
# unoisebase[:,3]=iid_noise_u['I'][:,1]/std_iidn_u[1,1]

# # v base
# vnoisebase=np.zeros((NE,4))
# vnoisebase[:,0]=iid_noise_v['E'][:,0]/std_iidn_v[0,0]
# vnoisebase[:,2]=iid_noise_v['I'][:,0]/std_iidn_v[1,0]#*0
# vnoisebase[:,3]=iid_noise_v['I'][:,1]/std_iidn_v[1,1]

# # u base
# unoisebase=np.zeros((NE,4))
# unoisebase[:,0]=iid_noise_u['E'][:,0]/std_iidn_u[0,0]
# unoisebase[:,1]=iid_noise_u['E'][:,1]/std_iidn_u[0,1]#*0
# unoisebase[:,2]=iid_noise_u['I'][:,0]/std_iidn_u[1,0]#*0
# unoisebase[:,3]=iid_noise_u['I'][:,1]/std_iidn_u[1,1]

# # v base
# vnoisebase=np.zeros((NE,4))
# vnoisebase[:,0]=iid_noise_v['E'][:,0]/std_iidn_v[0,0]
# vnoisebase[:,1]=iid_noise_v['E'][:,1]/std_iidn_v[0,1]#*0
# vnoisebase[:,2]=iid_noise_v['I'][:,0]/std_iidn_v[1,0]#*0
# vnoisebase[:,3]=iid_noise_v['I'][:,1]/std_iidn_v[1,1]

In [167]:
# creat noise base 2

noise_u_norm=noise_u.copy()
for i in range(2):
    noise_u_norm[:,i]/=std_noise_u[i,0]
noise_v_norm=noise_v.copy()
for i in range(2):
    noise_v_norm[:,i]/=std_noise_v[i,0]

# tilde noise uEI-vEI
tilde_noise = np.zeros((NE,4))
tilde_noise[:,:2]=noise_u_norm.copy()
tilde_noise[:,2:]=noise_v_norm.copy()

iid_noise = np.zeros((NE,8))
iid_noise[:,:4]=unoisebase.copy()
iid_noise[:,4:]=vnoisebase.copy()

# u noise correlation
# utildeU
unoisecov = np.zeros((2,4))
unoisecov = noise_u_norm.T@unoisebase
uvbasecov=noise_u_norm.T@vnoisebase
# utildeV
vnoisecov = np.zeros((2,4))
vnoisecov = noise_v_norm.T@vnoisebase
vubasecov=noise_v_norm.T@unoisebase

# PLOT FIGURE
# total overlap rho
noise_rho=tilde_noise.T@iid_noise
# print(noise_rho)
plt.figure()
plt.imshow(noise_rho**2,cmap='jet')
plt.clim([0.0,1.5])
plt.colorbar()
iid_noise_cov=iid_noise.T@iid_noise
# print(iid_noise_cov)
plt.figure()
plt.imshow(iid_noise_cov,cmap='jet')
plt.clim([0.0,1.5])
plt.colorbar()

# alphas are for u, the left singular vectors
alphau = np.zeros((2,2))
alphau[0,0]=np.sum(iid_noise_u['E'][:,0]*noise_u[:,0])/np.sum(iid_noise_u['E'][:,0]*iid_noise_u['E'][:,0]) # alphau--u^E_EE,
alphau[1,1]=np.sum(iid_noise_u['I'][:,1]*noise_u[:,1])/np.sum(iid_noise_u['I'][:,1]*iid_noise_u['I'][:,1]) # alphau--u^I_II,
alphau[0,1]=np.sum(iid_noise_u['E'][:,1]*noise_u[:,0])/np.sum(iid_noise_u['E'][:,1]*iid_noise_u['E'][:,1]) # alphau--u^E_EI,
alphau[1,0]=np.sum(iid_noise_u['I'][:,0]*noise_u[:,1])/np.sum(iid_noise_u['I'][:,0]*iid_noise_u['I'][:,0]) # alphau--u^I_IE
# alphau*=2.0

# betas are for v, the right singular vectors
betav = np.zeros((2,2))
betav[0,0]=np.sum(iid_noise_v['E'][:,0]*noise_v[:,0])/np.sum(iid_noise_v['E'][:,0]*iid_noise_v['E'][:,0]) # alphau0--v^E_EE, 1-SQUARE v^E_IE
betav[1,1]=np.sum(iid_noise_v['I'][:,1]*noise_v[:,1])/np.sum(iid_noise_v['I'][:,1]*iid_noise_v['I'][:,1]) # alphau0--v^I_II, 1-SQUARE V^I_EI
betav[0,1]=np.sum(iid_noise_v['E'][:,1]*noise_v[:,0])/np.sum(iid_noise_v['E'][:,1]*iid_noise_v['E'][:,1]) # alphau0--v^E_EE, 1-SQUARE v^E_IE
betav[1,0]=np.sum(iid_noise_v['I'][:,0]*noise_v[:,1])/np.sum(iid_noise_v['I'][:,0]*iid_noise_v['I'][:,0]) # alphau0--v^I_II, 1-SQUARE V^I_EI
# betav*=2.0

# for i in range(2):
#     if alphau[i,0]>1.0:
#         alphau[i,0]=1.0
#     if betav[i,0]>1.0:
#         betav[i,0]=1.0

In [168]:
print(eigvgm[1]/eigvgm[0])
print(alphau[0,0]*(xee/JEE)**2*JEE/(alphau[1,1]*(xii/JII)**2*JIE))
print(betav[0,0]*(xee/JEE)**2*JEE/(betav[1,1]*(xii/JII)**2*JEI))

(0.6944444444444444+0j)
0.6967483276886809
0.694887469967443


## check noise components rotation

In [169]:
noise_u_org,noise_v_org = eigvecG@noise_u.T,eigvecG@noise_v.T
noise_u_org=noise_u_org.T
noise_v_org=noise_v_org.T

In [170]:
# before rotation
ticks = np.linspace(-0.3,0.3,3)
lims= [-0.31,0.31]
fig,ax=plt.subplots(4,4,figsize=(9,9))
for i in range(2):
    for j in range(2):
        ax[i][j].scatter(noise_u[:,i]*np.sqrt(NE),noise_u[:,j]*np.sqrt(NE),s=5.0,c='blue',alpha=0.5)
        ax[i][j].scatter(noise_u_org[:,i]*np.sqrt(NE),noise_u_org[:,j]*np.sqrt(NE),s=5.0,c='red',alpha=0.5)
for i in range(2):
    for j in range(2):
        ax[2+i][2+j].scatter(noise_v[:,i]*np.sqrt(NE),noise_v[:,j]*np.sqrt(NE),s=5.0,c='blue',alpha=0.5)
        ax[2+i][2+j].scatter(noise_v_org[:,i]*np.sqrt(NE),noise_v_org[:,j]*np.sqrt(NE),s=5.0,c='red',alpha=0.5)
for i in range(2):
    for j in range(2):
        ax[i][2+j].scatter(noise_u[:,i]*np.sqrt(NE),noise_v[:,j]*np.sqrt(NE),s=5.0,c='blue',alpha=0.5)
        ax[i][2+j].scatter(noise_u_org[:,i]*np.sqrt(NE),noise_v_org[:,j]*np.sqrt(NE),s=5.0,c='red',alpha=0.5)
for i in range(2):
    for j in range(2):
        ax[2+i][j].scatter(noise_v[:,i]*np.sqrt(NE),noise_u[:,j]*np.sqrt(NE),s=5.0,c='blue',alpha=0.5)
        ax[2+i][j].scatter(noise_v_org[:,i]*np.sqrt(NE),noise_u_org[:,j]*np.sqrt(NE),s=5.0,c='red',alpha=0.5)
for i in range(4):
    for j in range(4):
        ax[i][j].set_xlim(lims)
        ax[i][j].set_ylim(lims)
        ax[i][j].set_xticks(ticks)
        ax[i][j].set_yticks(ticks)
        ax[i][j].set_aspect('equal')
        ax[i][j].grid()
# for i in range(4):
#     for j in range(4):
#         ax[i][j].set_xlim(lims)
#         ax[i][j].set_ylim(lims)
#         ax[i][j].set_xticks([])#ticks)
#         ax[i][j].set_yticks([])#ticks)
#         ax[i][j].set_aspect('equal')
#         ax[i][j].grid()
ax[0][3].set_xticks(ticks)
ax[0][3].set_yticks(ticks)
ax[3][0].set_xticks(ticks)
ax[3][0].set_yticks(ticks)
ax[0][1].set_xticks(ticks)
ax[0][1].set_yticks(ticks)
ax[1][0].set_xticks(ticks)
ax[1][0].set_yticks(ticks)

## check the overlap matrix in random/noise components

In [178]:
## zetau/vEI  prime
# Block Gaussian Random Network
xee,xei,xie,xii=0.3,0.2,0.2,0.6#1.2,1.2,1.8
JE,JI=9.0*xee,10.0*xii

JEE,JIE,JEI,JII=JE,JE,JI,JI


# generate random 
Grand=np.array([xee,xei,xie,xii])
X,Gamp,eigvgmT,eigvecgm,gaverage,ugm,svgm,vgm=generate_localstatsmat(Nparams,Grand)
eigvgm=eigvals(eigvgmT)
idx = eigvgm.argsort()[::-1]   
eigvgm = eigvgm[idx]
gradius=np.sqrt(eigvgm[0])
# generate mean Am
Am,Jsv,ua,svam,va=generate_meanmat(Nparams,JEE,JIE,JEI,JII)
Xtotal=X.copy()+Am.copy()
# Combine, total svd decomposition
uxt,svxt,vhxt=la.svd(Xtotal)
vxt=vhxt.T


In [179]:
IDX=0
noise_u_,noise_v_=np.zeros((NE,2)),np.zeros((NE,2))
noise_u_[:,0]= uxt[:NE,IDX]-np.mean(uxt[:NE,IDX])
noise_u_[:,1]= uxt[NE:,IDX]-np.mean(uxt[NE:,IDX])
noise_v_[:,0]= vxt[:NE,IDX]-np.mean(vxt[:NE,IDX])
noise_v_[:,1]= vxt[NE:,IDX]-np.mean(vxt[NE:,IDX])

In [176]:
print(noise_u_.T@noise_v_*NE)
print(noise_u.T@noise_v*NE)

[[-0.02931933 -0.04310275]
 [-0.06672983  0.00274242]]
[[ 0.01713054  0.02448348]
 [ 0.03106086 -0.00031125]]


In [186]:
fig,ax=plt.subplots(2,3,figsize=(12,6))
ax[0][0].scatter(noise_u_[:,0]*np.sqrt(NE),noise_u_[:,1]*np.sqrt(NE),s=5.0,c='blue',alpha=0.5)
ax[0][1].scatter(noise_u_[:,0]*np.sqrt(NE),noise_v_[:,0]*np.sqrt(NE),s=5.0,c='blue',alpha=0.5)
ax[0][2].scatter(noise_u_[:,0]*np.sqrt(NE),noise_v_[:,1]*np.sqrt(NE),s=5.0,c='blue',alpha=0.5)

ax[1][0].scatter(noise_u_[:,1]*np.sqrt(NE),noise_u_[:,0]*np.sqrt(NE),s=5.0,c='blue',alpha=0.5)
ax[1][1].scatter(noise_v_[:,0]*np.sqrt(NE),noise_u_[:,0]*np.sqrt(NE),s=5.0,c='blue',alpha=0.5)
ax[1][2].scatter(noise_v_[:,1]*np.sqrt(NE),noise_u_[:,0]*np.sqrt(NE),s=5.0,c='blue',alpha=0.5)

for i in range(2):
    for j in range(3):
        ax[i][j].set_xlim(lims)
        ax[i][j].set_ylim(lims)
        ax[i][j].set_xticks(ticks)
        ax[i][j].set_yticks(ticks)
        ax[i][j].set_aspect('equal')
        ax[i][j].grid()
# for i in range(4):
#     for j in range(4):
#         ax[i][j].set_xlim(lims)
#         ax[i][j].set_ylim(lims)
#         ax[i][j].set_xticks([])#ticks)
#         ax[i][j].set_yticks([])#ticks)
#         ax[i][j].set_aspect('equal')
#         ax[i][j].grid()
# ax[0][3].set_xticks(ticks)
# ax[0][3].set_yticks(ticks)
# ax[3][0].set_xticks(ticks)
# ax[3][0].set_yticks(ticks)
# ax[0][1].set_xticks(ticks)
# ax[0][1].set_yticks(ticks)
# ax[1][0].set_xticks(ticks)
# ax[1][0].set_yticks(ticks)

In [187]:
# before rotation
fig,ax=plt.subplots(4,4,figsize=(9,9))
for i in range(2):
    for j in range(2):
        # ax[i][j].scatter(noise_u_org[:,i]*np.sqrt(NE),noise_u_org[:,j]*np.sqrt(NE),s=5.0,c='red',alpha=0.05)
        ax[i][j].scatter(noise_u_[:,j]*np.sqrt(NE),noise_u_[:,i]*np.sqrt(NE),s=5.0,c='blue',alpha=0.5)
for i in range(2):
    for j in range(2):
        # ax[2+i][2+j].scatter(noise_v_org[:,i]*np.sqrt(NE),noise_v_org[:,j]*np.sqrt(NE),s=5.0,c='red',alpha=0.05)
        ax[2+i][2+j].scatter(noise_v_[:,j]*np.sqrt(NE),noise_v_[:,i]*np.sqrt(NE),s=5.0,c='blue',alpha=0.5)
for i in range(2):
    for j in range(2):
        # ax[i][2+j].scatter(noise_u_org[:,i]*np.sqrt(NE),noise_v_org[:,j]*np.sqrt(NE),s=5.0,c='red',alpha=0.05)
        ax[i][2+j].scatter(noise_v_[:,j]*np.sqrt(NE),noise_u_[:,i]*np.sqrt(NE),s=5.0,c='blue',alpha=0.5)
for i in range(2):
    for j in range(2):
        # ax[2+i][j].scatter(noise_v_org[:,i]*np.sqrt(NE),noise_u_org[:,j]*np.sqrt(NE),s=5.0,c='red',alpha=0.05)
        ax[2+i][j].scatter(noise_u_[:,j]*np.sqrt(NE),noise_v_[:,i]*np.sqrt(NE),s=5.0,c='blue',alpha=0.5)
for i in range(4):
    for j in range(4):
        ax[i][j].set_xlim(lims)
        ax[i][j].set_ylim(lims)
        ax[i][j].set_xticks(ticks)
        ax[i][j].set_yticks(ticks)
        ax[i][j].set_aspect('equal')
        ax[i][j].grid()
# for i in range(4):
#     for j in range(4):
#         ax[i][j].set_xlim(lims)
#         ax[i][j].set_ylim(lims)
#         ax[i][j].set_xticks([])#ticks)
#         ax[i][j].set_yticks([])#ticks)
#         ax[i][j].set_aspect('equal')
#         ax[i][j].grid()
ax[0][3].set_xticks(ticks)
ax[0][3].set_yticks(ticks)
ax[3][0].set_xticks(ticks)
ax[3][0].set_yticks(ticks)
ax[0][1].set_xticks(ticks)
ax[0][1].set_yticks(ticks)
ax[1][0].set_xticks(ticks)
ax[1][0].set_yticks(ticks)

In [1738]:
ticks = np.linspace(-2.0,2.0,5)
lims= [-2.1,2.1]
nbins=100
fig,ax=plt.subplots(1,2,figsize=(10,4.5))
# eigenvalue
ax[0].scatter(uxt[:NE,0],vxt[:NE,0],s=15.0,c='blue',alpha=0.5)
ax[0].scatter(utilde_rec[:,0],vtilde_rec[:,0],s=15.0,c='red',alpha=0.5)
ax[1].scatter(uxt[NE:,0],vxt[NE:,0],s=15.0,c='blue',alpha=0.5)
ax[1].scatter(utilde_rec[:,1],vtilde_rec[:,1],s=15.0,c='red',alpha=0.5)

fig,ax=plt.subplots(1,2,figsize=(10,4.5))
# eigenvalue
ax[0].scatter(uxt[:NE,0],vxt[:NE,0],s=15.0,c='blue',alpha=0.5)
ax[1].scatter(utilde_rec[:,0],vtilde_rec[:,0],s=15.0,c='blue',alpha=0.5)
ax[0].scatter(uxt[NE:,0],vxt[NE:,0],s=15.0,c='red',alpha=0.5)
ax[1].scatter(utilde_rec[:,1],vtilde_rec[:,1],s=15.0,c='red',alpha=0.5)

In [1230]:
print(xee,xei,xie,xii)
print('--------------------')
print('mean value of u with E type (EE EI):')
print(iid_mu_u['E'],iid_mu_u['E'][0,0]/iid_mu_u['E'][1,0])
print('mean UE for total')
print(np.sqrt(1-(xee/JE)**2)/np.sqrt(NE),np.sqrt(1-(xei/JI)**2)/np.sqrt(NE),(np.sqrt(1-(xee/JE)**2)/np.sqrt(1-(xei/JI)**2)))

print('mean value of u with I type (IE II):')
print(iid_mu_u['I'],iid_mu_u['I'][0,0]/iid_mu_u['I'][1,0])
print('mean UI for total')
print(np.sqrt(1-(xie/JE)**2)/np.sqrt(NE),np.sqrt(1-(xii/JI)**2)/np.sqrt(NE),(np.sqrt(1-(xie/JE)**2)/np.sqrt(1-(xii/JI)**2)))

print('mean value of v with E type (EE IE):')
print(iid_mu_v['E'],iid_mu_v['E'][0,0]/iid_mu_v['E'][1,0])
print('mean VE for total')
print(np.sqrt(1-(xee/JE)**2)/np.sqrt(NE),np.sqrt(1-(xie/JE)**2)/np.sqrt(NE),(np.sqrt(1-(xee/JE)**2)/np.sqrt(1-(xie/JE)**2)))

print('mean value of v with I type (EI II):')
print(iid_mu_v['I'],iid_mu_v['I'][0,0]/iid_mu_v['I'][1,0])
print('mean VI for total')
print(np.sqrt(1-(xei/JI)**2)/np.sqrt(NE),np.sqrt(1-(xii/JI)**2)/np.sqrt(NE),(np.sqrt(1-(xei/JI)**2)/np.sqrt(1-(xii/JI)**2)))

2.0 0.0 0.0 1.0
--------------------
mean value of u with E type (EE EI):
[[-0.01803964]
 [-0.03162278]] 0.5704633770780799
mean UE for total
0.018973665961010272 0.03162277660168379 0.5999999999999999
mean value of u with I type (IE II):
[[-0.03162278]
 [-0.03036734]] 1.0413418318129657
mean UI for total
0.03162277660168379 0.030304576336566323 1.0434983894999017
mean value of v with E type (EE IE):
[[-0.01804564]
 [-0.03162278]] 0.5706532412385339
mean VE for total
0.018973665961010272 0.03162277660168379 0.5999999999999999
mean value of v with I type (EI II):
[[0.03162278]
 [0.03034521]] 1.0421010282814995
mean VI for total
0.03162277660168379 0.030304576336566323 1.0434983894999017


## Figure out parameters

In [972]:
# First Set of Figures
# figure, validate results in sharpee's paper
# ticks and labels
ticks = np.linspace(-2.0,2.0,5)
lims= [-2.1,2.1]
nbins=100
fig,ax=plt.subplots(1,2,figsize=(10,4.5))
# eigenvalue
ax[0].scatter(np.real(eigvx),np.imag(eigvx),s=2.0,c='blue',alpha=0.5)
# ax[0].set_xlim(lims)
# ax[0].set_ylim(lims)
# ax[0].set_xticks(ticks)
# ax[0].set_yticks(ticks)
ax[0].set_aspect('equal')

# back up code

In [ ]:
# Block Gaussian Random Network
xee,xei,xie,xii=0.6,0.25,0.25,0.3#1.2,1.2,1.8
JE,JI=5.0*xee,4.0*xii

# generate random 
Grand=np.array([xee,xei,xie,xii])
X,Gamp,eigvgmT,eigvecgm,gaverage,ugm,svgm,vgm=generate_localstatsmat(Nparams,Grand)
eigvgm=eigvals(eigvgmT)
idx = eigvgm.argsort()[::-1]   
eigvgm = eigvgm[idx]
gradius=np.sqrt(eigvgm[0])
# generate mean Am
Am,Jsv,ua,svam,va=generate_meanmat(Nparams,JE,JI)
# # CALCULATE XTOTAL
# Xtotal=X.copy()+Am.copy()
# # Combine, total svd decomposition
# uxt,svxt,vhxt=la.svd(Xtotal)
# vxt=vhxt.T   
# eigvx,eigvecx=la.eig(Xtotal)

In [1218]:
# calculate mean for real
mu_utilde = np.zeros((2,1))
mu_utilde[0,0],mu_utilde[1,0]=np.mean(uxt[:NE,0]),np.mean(uxt[NE:,0])
noise_utilde = np.zeros((2,1))
noise_utilde[0,0],noise_utilde[1,0]=np.std(uxt[:NE,0]),np.std(uxt[NE:,0])

# vector to calculate uE-real
angle_1= np.arctan2(xee/JE/np.sqrt(NE),-np.sqrt(1-(xee/JE)**2)/np.sqrt(NE))
angle_2 = np.arctan2(xei/JI/np.sqrt(NE),-np.sqrt(1-(xei/JI)**2)/np.sqrt(NE))
vec_1=np.array([[np.cos(angle_1),np.sin(angle_1)]]).T
vec_1/=np.sqrt(NE)
print('vec1',vec_1,vec_1[0]/vec_1[1])
print('numvec1:',iid_mu_u['E'][0,0],np.std(uxtEE[:,0]),iid_mu_u['E'][0,0]/np.std(uxtEE[:,0]))
vec_2=np.array([[np.cos(angle_2),np.sin(angle_2)]]).T
vec_2/=np.sqrt(NE)
# print('vec2',vec_2,vec_2[0]/vec_2[1])
# print('numvec2:',iid_mu_u['E'][1,0],np.std(uxtEI[:,0]),iid_mu_u['E'][1,0]/np.std(uxtEI[:,0]))
# print('angles')
# print(angle_1,angle_2)
rot_theta = (angle_1-angle_2)/2.0
Rot1=np.array([[np.cos(rot_theta),-np.sin(rot_theta)],[np.sin(rot_theta),np.cos(rot_theta)]])
Rot2=np.array([[np.cos(-rot_theta),-np.sin(-rot_theta)],[np.sin(-rot_theta),np.cos(-rot_theta)]])
# the first is gonna be drew back 
# the second is gonna be pushed forward

theo_mu=np.zeros((2,2))
theo_mu[0,:]= np.squeeze(Rot2@vec_1)*xee/JE
test_ang2= Rot1@vec_2*xei/JI
# print('rotate')
# print(theo_mu[0,:],test_ang2)
# print(np.sqrt(theo_mu[0,1]**2+test_ang2[1]**2))
print('real mean:')
print(mu_utilde[0,0],noise_utilde[0,0])
# print(mu_utilde[0,0],iid_mu_u['E'][0,0]/np.sqrt(xee/JE),iid_mu_u['E'][1,0]/np.sqrt(xei/JI))#,noise_utilde[0,0])
# print('noise comp:',noise_utilde[0,0]/vec_1[1])

# vector to calculate uE-real
angle_1= np.arctan2(xie/JE/np.sqrt(NE),np.sqrt(1-(xie/JE)**2)/np.sqrt(NE))
angle_2 = np.arctan2(xii/JI/np.sqrt(NE),-np.sqrt(1-(xii/JI)**2)/np.sqrt(NE))
vec_1=np.array([[np.cos(angle_1),np.sin(angle_1)]]).T
vec_1/=np.sqrt(NE)
# print('vec1',vec_1,vec_1[0]/vec_1[1])
# print('numvec1:',iid_mu_u['I'][0,0],np.std(uxtIE[:,0]),iid_mu_u['I'][0,0]/np.std(uxtIE[:,0]))
vec_2=np.array([[np.cos(angle_2),np.sin(angle_2)]]).T
vec_2/=np.sqrt(NE)
print('vec2',vec_2,vec_2[0]/vec_2[1])
print('numvec2:',iid_mu_u['I'][1,0],np.std(uxtII[:,0]),iid_mu_u['I'][1,0]/np.std(uxtII[:,0]))
# print('angles')
# print(angle_1,angle_2)
rot_theta = (angle_1-angle_2)/2.0
Rot1=np.array([[np.cos(rot_theta),-np.sin(rot_theta)],[np.sin(rot_theta),np.cos(rot_theta)]])
Rot2=np.array([[np.cos(-rot_theta),-np.sin(-rot_theta)],[np.sin(-rot_theta),np.cos(-rot_theta)]])
# the first is gonna be drew back 
# the second is gonna be pushed forward

theo_mu=np.zeros((2,2))
theo_mu[0,:]= np.squeeze(Rot2@vec_1)*xee/JE
test_ang2= Rot1@vec_2*xei/JI
# print('rotate')
# print(theo_mu[0,:],test_ang2)
# print(np.sqrt(theo_mu[0,1]**2+test_ang2[1]**2))
print('real mean:')
print(mu_utilde[1,0],noise_utilde[1,0])
# print('noise comp:',noise_utilde[1,0]/vec_2[1])




vec1 [[-0.01897367]
 [ 0.02529822]] [-0.75]
numvec1: -0.01947075138589678 0.024917661215864535 -0.7814036484892962
real mean:
0.02055340669452231 0.009104648046617814
vec2 [[-0.01897367]
 [ 0.02529822]] [-0.75]
numvec2: 0.0172223163447693 0.02652153501818273 0.6493710236968548
real mean:
0.020406871667918793 0.008844345420995098


In [ ]:
# print(iid_sv)

overlaptt,overlaptheo,overlaptheo2=np.zeros((2,2)),np.zeros((2,2)),np.zeros((2,2))
overlaptt[0,0]=np.sum(uxt[:NE,0]*vxt[:NE,0]*svxt[0])
print(np.mean(uxt[:NE,0]),np.mean(vxt[:NE,0]),svxt[0])
overlaptt[0,1]=np.sum(uxt[:NE,0]*vxt[NE:,0]*svxt[0])
print(np.mean(uxt[:NE,0]),np.mean(vxt[NE:,0]),svxt[0])
overlaptt[1,0]=np.sum(uxt[NE:,0]*vxt[:NE,0]*svxt[0])
overlaptt[1,1]=np.sum(uxt[NE:,0]*vxt[NE:,0]*svxt[0])

overlaptheo[0,0]=np.sum(uxtEE[:,0]*vxtEE[:,0]*svxtEE[0])#iid_sv[0,0]*(1-xee**2/JE**2)
print(np.mean(uxtEE[:,0]),np.mean(vxtEE[:,0]),svxtEE[0])
overlaptheo[0,1]=np.sum(uxtEI[:,0]*vxtEI[:,0]*svxtEI[0])#iid_sv[0,1]*(1-xei**2/JI**2)
print(np.mean(uxtEI[:,0]),np.mean(vxtEI[:,0]),svxtEI[0])
overlaptheo[1,0]=np.sum(uxtIE[:,0]*vxtIE[:,0]*svxtIE[0])#iid_sv[1,0]*(1-xie**2/JE**2)
overlaptheo[1,1]=np.sum(uxtII[:,0]*vxtII[:,0]*svxtII[0])#iid_sv[1,1]*(1-xii**2/JI**2)

overlaptheo2[0,0]=np.sum(iid_sv[0,0]*(1-xee**2/JE**2))
overlaptheo2[0,1]=-np.sum(iid_sv[0,1]*(1-xei**2/JI**2))
overlaptheo2[1,0]=np.sum(iid_sv[1,0]*(1-xie**2/JE**2))
overlaptheo2[1,1]=-np.sum(iid_sv[1,1]*(1-xii**2/JI**2))

print('tilde overlap:')
print(overlaptt)
print('theo overlap:')
print(overlaptheo)
print('theo overlap2:')
print(overlaptheo2)
print('change:')
overlapchange=overlaptheo2-overlaptt
print(overlapchange)
# eigvtt,eigvectt=la.eig(overlaptt)
# eigvtheo,eigvectheo=la.eig(overlaptheo)
# print('eigenvalue compare:')
# print(eigvtt)
# print(eigvtheo)

In [879]:
Xtotal=X.copy()+Am.copy()
# singular decomposition
uxt,svxt,vhxt=la.svd(Xtotal)
vxt=vhxt.T
ux,svx,vhx=la.svd(X)
vx=vhx.T
print(svxt[:5])
print(svx[:5])


## dissect noise and mean components
## mean for total and original
muxtt,mvxtt=np.zeros((2,1)),np.zeros((2,1))
muxtt[0,0],muxtt[1,0]=np.mean(uxt[:NE,0]),np.mean(uxt[NE:,0])
mvxtt[0,0],mvxtt[1,0]=np.mean(vxt[:NE,0]),np.mean(vxt[NE:,0])
mua,mva = np.zeros((2,1)),np.zeros((2,1))
mua[0,0],mua[1,0]=ua[0,0],ua[NE+1,0]
mva[0,0],mva[1,0]=va[0,0],va[NE+1,0]
# noise components E and I seperate 
noiseu,noisev=np.zeros((NE,2)),np.zeros((NE,2))
# for E noise
noiseu[:,0]=(uxt[:NE,0]-muxtt[0,0])
noisev[:,0]=(vxt[:NE,0]-mvxtt[0,0])
# for I noise
noiseu[:,1]=(uxt[NE:,0]-muxtt[1,0])
noisev[:,1]=(vxt[NE:,0]-mvxtt[1,0])
noisem,noisen=noiseu.copy(),noisev.copy()#*svxt[0]

# noise components 4*4
noisevec = np.zeros((NE,4))
noisevec[:,:2]=noisem.copy()
noisevec[:,2:]=noisen.copy()


[6.44943161 4.00928745 3.98429257 3.96936678 3.94801228]
[4.00973264 3.98515474 3.97197553 3.94868445 3.92199661]


In [800]:
mleft=5.0*np.random.randn(NE,1)/np.sqrt(NE)
nright=4.0*np.random.randn(NE,1)/np.sqrt(NE)
Cnright = 3*np.ones((NE,1))
Cmleft = 1*np.ones((NE,1))
randomm=mleft@Cnright.T+Cmleft@nright.T 
lrand=np.reshape(randomm,(NE**2,1))
theostd = (5*3)**2/NE+(4*1)**2/NE
theostd = np.sqrt(theostd)
print('stdnum:',np.std(lrand),theostd)

stdnum: 0.4973373839263357 0.4909175083453431


In [ ]:
# # subject to
# sigmE,signI,sigmI,signE=2.0,3.0,1.0,4.0
# rhoE=xee/sigmE/signE*1.5
# rhoI=xii/sigmI/signI*1.5
# m,n=np.zeros((N,1)),np.zeros((N,1))
# zetaxE,zetayE,zetaxI,zetayI=np.random.randn(NE),np.random.randn(NE),np.random.randn(NI),np.random.randn(NI)
# m[:NE,0]=sigmE*zetaxE/np.sqrt(N)
# n[:NE,0]=signE*(rhoE*zetaxE+np.sqrt(1-rhoE**2)*zetayE)/np.sqrt(N)
# m[NE:,0]=sigmI*zetaxI/np.sqrt(N)
# n[NE:,0]=signI*(rhoI*zetaxI+np.sqrt(1-rhoI**2)*zetayI)/np.sqrt(N)
# P=m@n.T 
# Xtilde=X.copy()+P.copy()

In [284]:
stdme=np.std(lxvec[:NE,0])
stdmi=np.std(lxvec[NE:,0])
stdne=np.std(svx[0]*rxvec[:NE,0])
stdni=np.std(svx[0]*rxvec[NE:,0])
rm1=np.array([stdme,stdmi])
rm1=np.reshape(rm1,(2,1))
rn1=np.array([stdne,stdni])
rn1=np.reshape(rn1,(2,1))
stdme=np.std(lxvec[:NE,1])
stdmi=np.std(lxvec[NE:,1])
stdne=np.std(svx[1]*rxvec[:NE,1])
stdni=np.std(svx[1]*rxvec[NE:,1])
rm2=np.array([stdme,stdmi])
rm2=np.reshape(rm2,(2,1))
rn2=np.array([stdne,stdni])
rn2=np.reshape(rn2,(2,1))
gstdest=rm1@rn1.T +rm2@rn2.T 
print('cal g:',gstdest)
print('org g:',xee**2/N,xei**2/N,xie**2/N,xii**2/N)

cal g: [[0.00089115 0.00017327]
 [0.00276879 0.00053832]]
org g: 0.000625 0.0009 0.0025 0.000225


In [269]:
lxvec,svx,rxvech=la.svd(P)
rxvec=rxvech.T 
fig,ax=plt.subplots(2,2,figsize=(9,9))
nbins=30
n, bins, patches = ax[0][0].hist(lxvec[:NE,0], nbins, density=True, alpha=0.5)
n, bins, patches = ax[1][0].hist(lxvec[NE:,0], nbins, density=True, alpha=0.5)
n, bins, patches = ax[0][1].hist(rxvec[:NE,0], nbins, density=True, alpha=0.5)
n, bins, patches = ax[1][1].hist(rxvec[NE:,0], nbins, density=True, alpha=0.5)
